<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AI_Agent_Tutorial_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.deepseek.com/usage

In [ ]:
!pip install langchain_openai colab-env -q

In [ ]:
import os
import colab_env

In [11]:
from google.colab import userdata
from litellm import completion
import os

deepseek_api_key_test = userdata.get('DEEPSEEK_API_KEY')
#print(f"Test Key: {deepseek_api_key_test}")

try:
    response = completion(
        model="deepseek/deepseek-reasoner",
        messages=[{"role": "user", "content": "hello"}],
        api_key=deepseek_api_key_test # Explicitly pass the key
    )
    print("Test call successful!")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Test call failed: {e}")

Test call successful!
Hello! 👋 How can I help you today? 😊


In [8]:
!pip install crewai -q # Install CrewAI
!pip install crewai-tools -q # Install CrewAI Tools
!pip install 'crewai[tools]' -q

!pip install flask_login -q
!pip install openai -q

CrewAI Demonstration Explanation
This CrewAI implementation addresses the five points from your "Building AI Agents From Scratch" series:

1. What AI Agents Are:
In CrewAI, agents are instantiated using the Agent class. Each agent is given a role, a goal, and a backstory. These attributes collectively define the agent's persona and purpose, guiding its decision-making and interaction within the crew. For instance, our InformationGatherer agent is designed specifically for factual data retrieval, while the CreativeContentCreator focuses on narrative generation.

2. How Tool Usage in AI Agents Actually Works:
CrewAI seamlessly integrates standard Python functions as tools. You simply define a function (e.g., get_current_weather, get_stock_price, create_story) and then pass it directly into the tools list when initializing an Agent. The underlying LLM (like Gemini) is then responsible for understanding the tool's purpose (from its docstring) and parameters (from its signature) and deciding when and how to call it. The print statements within each tool function clearly show when a tool is being invoked by the agent.

3. How to Build a Decorator Wrapper that Extracts Relevant Details from a Python Function:
CrewAI simplifies tool definition. Instead of a custom decorator to extract function details, CrewAI's Agent class expects callable functions directly. The framework and the LLM handle the interpretation of the function's docstring and signature for tool description and parameter schema. This abstracts away the need for a manual decorator wrapper, as the LLM's capabilities are leveraged to understand the function's utility.

4. How to Think About Constructing Effective System Prompts:
CrewAI excels here by allowing you to define the agent's "system prompt" through its role, goal, and backstory parameters. These descriptive fields implicitly form a comprehensive prompt that guides the LLM attached to the agent. For example, the InformationGatherer's backstory tells the LLM that this agent is "diligent and precise" and "excellent at understanding user queries and identifying the correct tool." Tasks also include a description that further refines the specific objective for the agent.

5. How to Build an Agent Class That is Able to Plan and Execute Actions Using Provided Tools:
CrewAI provides the Crew class for orchestrating agents and their tasks. You define Task objects, assigning them to specific agents. The crew.kickoff() method initiates the process, where the agents, guided by their roles and the task descriptions, "plan" (their internal LLM reasoning) and "execute" by calling the appropriate tools. The verbose=2 setting in both Agent and Crew helps you observe this planning and execution flow in detail.

This demonstration provides a robust framework for building more complex AI agents, directly transferable to our flight planning AI agent project, where different agents could handle weather, air traffic control regulations, route optimization, and passenger communications.

Let me know if you'd like to explore how to integrate specific real-world APIs with these tools!

In [23]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool, tool
from langchain_community.chat_models import ChatLiteLLM # For direct LiteLLM integration
from pydantic import BaseModel, Field # For defining tool arguments

## BEGIN deepseek API
from google.colab import userdata
from litellm import completion # Imported for direct API key testing

# IMPORTANT: Ensure DEEPSEEK_API_KEY is correctly set in your Google Colab userdata secrets.
# This error (AuthenticationError: DeepseekException - Authentication Fails) indicates a problem
# with the API key itself or its validity on the Deepseek side.
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
# print(f"DEEPSEEK_API_KEY loaded: {'*' * len(deepseek_api_key) if deepseek_api_key else 'Not set'}") # For debugging purposes

if not deepseek_api_key:
    raise ValueError("DEEPSEEK_API_KEY not found in Google Colab userdata. Please set it to proceed.")

# Set the model name for Deepseek
model_name = "deepseek-reasoner"

# --- LLM SETUP for CrewAI with Deepseek ---
# CrewAI expects a Langchain LLM object. We now use Langchain's ChatLiteLLM
# for a more direct integration with LiteLLM, which handles the Deepseek provider.
llm = ChatLiteLLM(
    model=f"deepseek/{model_name}", # Correct format for LiteLLM to specify provider and model
    api_key=deepseek_api_key,       # LiteLLM uses api_key directly
    api_base="https://api.deepseek.com" # Ensure the base URL is correct
)
## END deepseek API


# --- 2. How Tool Usage in AI Agents Actually Works (CrewAI Style with BaseTool) ---
# In CrewAI, tools can be standard Python functions, but for more control
# and explicit argument definition, you can use BaseTool. This allows
# you to define a Pydantic schema for the tool's arguments.

# Define the argument schema for get_current_weather
class GetCurrentWeatherArgsSchema(BaseModel):
    location: str = Field(description="The city name for which to get the weather.")
    unit: str = Field(description="The unit of temperature, either 'celsius' or 'fahrenheit'. Defaults to 'celsius'.", default="celsius")

class GetCurrentWeatherTool(BaseTool):
    name: str = "get_current_weather"
    description: str = "Retrieves the current weather conditions for a specific city. Use this when the user asks for weather information."
    args_schema: type[BaseModel] = GetCurrentWeatherArgsSchema

    def _run(self, location: str, unit: str = "celsius") -> str:
        """
        Retrieves the current weather conditions for a given city.
        """
        print(f"  [Tool Call] get_current_weather(location='{location}', unit='{unit}')")
        # In a real application, this would call an external weather API.
        # For demo purposes, we return a mock value.
        if location.lower() == "montreal":
            return f"The current weather in Montreal is 20°{unit.upper()} and sunny."
        elif location.lower() == "london":
            return f"The current weather in London is 15°{unit.upper()} and cloudy."
        elif location.lower() == "toronto": # Added Toronto to the mock data
            return f"The current weather in Toronto is 22°{unit.upper()} and partly cloudy."
        else:
            return f"Could not find weather for {location}. Please try a major city."

# Define the argument schema for get_stock_price
class GetStockPriceArgsSchema(BaseModel):
    ticker: str = Field(description="The company ticker symbol (e.g., 'GOOG' for Google, 'AAPL' for Apple).")

class GetStockPriceTool(BaseTool):
    name: str = "get_stock_price"
    description: str = "Retrieves the current stock price for a given company ticker symbol. Use this when the user asks for stock prices."
    args_schema: type[BaseModel] = GetStockPriceArgsSchema

    def _run(self, ticker: str) -> float:
        """
        Retrieves the current stock price for a specified ticker symbol.
        """
        print(f"  [Tool Call] get_stock_price(ticker='{ticker}')")
        # In a real application, this would call a stock market API.
        # For demo purposes, we return a mock value.
        ticker = ticker.upper()
        if ticker == "GOOG":
            return 175.50
        elif ticker == "AAPL":
            return 190.25
        else:
            return 0.0

# Define the argument schema for create_story
class CreateStoryArgsSchema(BaseModel):
    topic: str = Field(description="The topic or theme for the story (e.g., 'a brave knight' or 'space exploration').")

class CreateStoryTool(BaseTool):
    name: str = "create_story"
    description: str = "Generates a short creative story based on a provided topic. Use this when the user asks for a story."
    args_schema: type[BaseModel] = CreateStoryArgsSchema

    def _run(self, topic: str) -> str:
        """
        Generates a short creative story based on the provided topic.
        """
        print(f"  [Tool Call] create_story(topic='{topic}')")
        # In a real application, this would involve an LLM call or a more complex generation.
        # For demo purposes, we return a mock value.
        if "knight" in topic.lower():
            return f"Once upon a time, a brave knight set out on a quest to defeat a fearsome dragon in the {topic} realm. His sword gleamed under the moonlight as he approached the beast's lair."
        elif "space" in topic.lower():
            return f"In the vastness of space, a lone explorer discovered a new planet, teeming with {topic}. It was a world unlike any other, filled with crystalline flora and singing rivers."
        else:
            return f"Here's a story about {topic}: It was a dark and stormy night, and suddenly, a peculiar event unfolded, changing everything for the inhabitants of a small, quiet town."

# --- 3. Building a Decorator Wrapper (CrewAI uses direct assignment or BaseTool classes) ---
# With `BaseTool` classes, the explicit argument definition is handled by Pydantic's BaseModel
# and Field within the `args_schema`. This is a more robust and explicit way to define tools
# compared to just docstrings.

# --- 4. How to Think About Constructing Effective System Prompts (CrewAI Roles) ---
# CrewAI allows you to define explicit `role`, `goal`, and `backstory` for each agent.
# These act as the agent's system prompt, guiding its behavior and tool usage.

# Define our agents, each with a specific role and access to relevant tools
information_gatherer = Agent(
    role='Information Gatherer',
    goal='Accurately retrieve specific factual data from external sources, or answer general knowledge questions.',
    backstory="""You are a diligent and precise AI assistant specializing in factual data
    retrieval. You are excellent at understanding user queries and identifying the
    correct tool and parameters to fetch the requested information, such as weather
    conditions or stock prices. You can also answer general knowledge questions directly
    if no specific tool is applicable. Your descriptions of tools are crucial for the LLM.""",
    verbose=False, # Set to False to reduce verbose output
    allow_delegation=False, # For simplicity in this demo
    tools=[GetCurrentWeatherTool(), GetStockPriceTool()], # Pass instances of BaseTool classes
    llm=llm # Assign the LLM to the agent
)

creative_content_creator = Agent(
    role='Creative Content Creator',
    goal='Generate engaging and imaginative narratives based on user prompts.',
    backstory="""You are a highly imaginative AI, skilled at crafting compelling stories
    across various genres. You excel at taking a simple topic and expanding it into a
    rich, detailed narrative, always aiming to captivate your audience. Your descriptions of tools are crucial for the LLM.""",
    verbose=False, # Set to False to reduce verbose output
    allow_delegation=False, # For simplicity in this demo
    tools=[CreateStoryTool()], # Pass an instance of BaseTool class
    llm=llm # Assign the LLM to the agent
)

# --- 5. How to Build an Agent Class That is Able to Plan and Execute Actions Using Provided Tools (CrewAI) ---
# CrewAI's `Task` and `Crew` classes manage the planning and execution.
# Tasks define what needs to be done, and the Crew orchestrates the agents
# to complete these tasks.

def run_crew_demo(user_query: str):
    print(f"\n===== CrewAI Processing Request for: \"{user_query}\" =====")

    # Define tasks dynamically based on the user query
    tasks = []
    response_message = ""

    user_query_lower = user_query.lower()

    # Route queries based on keywords to either the Information Gatherer or Creative Content Creator.
    # The LLM within the agent will then decide which specific tool to use and extract arguments.
    if any(keyword in user_query_lower for keyword in ["weather", "temperature", "temp", "forecast", "stock", "price", "ticker"]):
        tasks.append(
            Task(
                description=f"Analyze the user's request: '{user_query}' and use the appropriate tool to retrieve the requested information. Be precise and provide the exact data requested, extracting all necessary parameters from the query.",
                agent=information_gatherer,
                expected_output=f"A concise answer to the user's query about weather or stock, including the relevant data.",
            )
        )
        response_message = f"I'm processing your request using the Information Gatherer agent."
    elif any(keyword in user_query_lower for keyword in ["story", "write", "tell me"]):
        tasks.append(
            Task(
                description=f"Analyze the user's request: '{user_query}' and use the create_story tool to generate a creative story. Ensure the story is engaging and directly addresses the topic identified in the user's request.",
                agent=creative_content_creator,
                expected_output=f"A creative and engaging story based on the user's specific request.",
            )
        )
        response_message = f"I'm generating a story for you using the Creative Content Creator agent."
    else:
        # If no specific tool task is matched, create a general task for the information_gatherer
        # so the LLM can attempt to answer it directly or state its limitations.
        tasks.append(
            Task(
                description=f"Answer the following question directly: '{user_query}'. If the question requires information not available via your tools and is general knowledge, answer it. Otherwise, state that you cannot assist with the query and that it is outside your current capabilities.",
                agent=information_gatherer,
                expected_output=f"A direct answer to the question or a clear statement of inability to answer.",
            )
        )
        response_message = f"I'm processing your general query using the Information Gatherer agent."


    # The 'if not tasks' block is no longer needed here because an 'else' task is always added.
    # This ensures the Crew always has at least one task to run.

    # Create a Crew and kick it off
    crew = Crew(
        agents=[information_gatherer, creative_content_creator],
        tasks=tasks,
        process=Process.sequential, # Agents execute tasks sequentially
        verbose=False, # Outputs only the final result of the crew.
    )

    try:
        result = crew.kickoff()
        return f"CrewAI final result for '{user_query}':\n{result}"
    except Exception as e:
        return f"An error occurred during CrewAI execution: {e}"


# --- Demonstration ---
if __name__ == "__main__":
    # Test your Deepseek API key directly first
    print("\n--- Testing Deepseek API Key Directly ---")
    deepseek_api_key_test = userdata.get('DEEPSEEK_API_KEY')
    # print(f"Test Key: {deepseek_api_key_test}") # Uncomment to see the key being loaded

    try:
        response = completion(
            model="deepseek/deepseek-reasoner", # Correct model format for LiteLLM
            messages=[{"role": "user", "content": "hello"}],
            api_key=deepseek_api_key_test # Explicitly pass the key
        )
        print("Test call successful! Response:")
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"Test call failed: {e}")
        print("Please ensure your DEEPSEEK_API_KEY is correct and active in Google Colab's userdata secrets.")
        # Exit if the API key test fails, as the rest of the demo won't work
        exit()

    print("\n--- Running CrewAI Demos (Requires successful API key test above) ---")
    print("\n--- Demo 1: Get weather for Montreal ---")
    print(run_crew_demo("What's the weather like in Montreal?"))

    print("\n\n--- Demo 2: Get weather for London in Fahrenheit ---")
    print(run_crew_demo("How about the temperature in London in fahrenheit?"))

    print("\n\n--- Demo 3: Get stock price for Google ---")
    print(run_crew_demo("What's the stock price of Google?"))

    print("\n\n--- Demo 4: Get stock price for Apple ---")
    print(run_crew_demo("Can you tell me the price of Apple stock?"))

    print("\n\n--- Demo 5: Create a story ---")
    print(run_crew_demo("Tell me a story about a brave knight."))

    print("\n\n--- Demo 6: Create a space exploration story ---")
    print(run_crew_demo("Write a story about space exploration."))

    print("\n\n--- Demo 7: Query not covered by tools ---")
    print(run_crew_demo("What is the capital of France?"))

    # MODIFIED: Demo 8 now requests weather for Toronto (a valid city in mock data)
    print("\n\n--- Demo 8: Get weather for Toronto ---")
    print(run_crew_demo("Get me the weather for Toronto."))



--- Testing Deepseek API Key Directly ---
Test call successful! Response:
Hello! 😊 How can I assist you today? Whether you have a question, need help with something, or just want to chat—I'm here for you!

--- Running CrewAI Demos (Requires successful API key test above) ---

--- Demo 1: Get weather for Montreal ---

===== CrewAI Processing Request for: "What's the weather like in Montreal?" =====
  [Tool Call] get_current_weather(location='Montreal', unit='celsius')
CrewAI final result for 'What's the weather like in Montreal?':
The current weather in Montreal is 20°C and sunny.


--- Demo 2: Get weather for London in Fahrenheit ---

===== CrewAI Processing Request for: "How about the temperature in London in fahrenheit?" =====
  [Tool Call] get_current_weather(location='London', unit='fahrenheit')
CrewAI final result for 'How about the temperature in London in fahrenheit?':
The current temperature in London is 15°F.


--- Demo 3: Get stock price for Google ---

===== CrewAI Processi